In [1]:
import pygtfs

In [2]:
sched = pygtfs.Schedule(":memory:")        # create a schedule object (a sqlite database)

In [3]:
pygtfs.append_feed(sched, r'../../data/raw/google_transit.zip')

Loading GTFS data for <class 'pygtfs.gtfs_entities.Agency'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Stop'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Transfer'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Route'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Fare'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.FareRule'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.ShapePoint'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Service'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.ServiceException'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Trip'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Frequency'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.StopTime'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.FeedInfo'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Translation'>:
1 record read for <class 'pygtfs.gtfs_entities.Agency'>.
1744 records read for <class 'pygtfs.gtfs_entities.Stop'>.
42 rec

In [16]:
import pandas as pd
import geopandas as gpd

In [18]:
service_ids = pd.read_csv('trips.txt')['service_id'].unique()
service_ids = frozenset(tuple(service_ids))

In [4]:
sched.agencies

[<Agency SATCo: PVTA>]

In [5]:
sched.routes

[<Route 30: 30>,
 <Route 31: 31>,
 <Route 33: 33>,
 <Route 36: 36>,
 <Route 45: 45>,
 <Route 46: 46>,
 <Route 801: T1>,
 <Route 802: T2>,
 <Route 803: T3>,
 <Route 804: T4>,
 <Route 805: T5>,
 <Route 806: T6>,
 <Route 820: T20>,
 <Route 821: T21>,
 <Route 823: T23>,
 <Route 824: T24>,
 <Route 892: T92>,
 <Route 921: P21E>,
 <Route B12: B12>,
 <Route B17: B17>,
 <Route B23: B23>,
 <Route B4: B4>,
 <Route B43: B43>,
 <Route B48: B48>,
 <Route B6: B6>,
 <Route B7: B7>,
 <Route G1: G1>,
 <Route G2: G2>,
 <Route G3: G3>,
 <Route G5: G5>,
 <Route G73: G73E>,
 <Route P20: P20>,
 <Route P21: P21>,
 <Route R10: R10>,
 <Route R14: R14>,
 <Route R24: R24>,
 <Route R29: R29>,
 <Route R41: R41>,
 <Route R42: R42>,
 <Route R44: R44>,
 <Route X90: X90>,
 <Route X92: X92>]

In [6]:
sched.stops

[<Stop 1: South Deerfield Center>,
 <Stop 10: Silver Lane>,
 <Stop 100: Haydenville Center OB>,
 <Stop 1000: Plainfield / Wason>,
 <Stop 10006: MassMutual Enfield>,
 <Stop 1001: Bay / Aster>,
 <Stop 1002: Springfield / Medical Ctr Drive (across)>,
 <Stop 1003: Carew / Van Horn>,
 <Stop 1005: Liberty / Cleveland>,
 <Stop 1006: Springfield / Medical Ctr Drive>,
 <Stop 1007: 3300 Main Street>,
 <Stop 1008: Carew/Van Horn(opposite)>,
 <Stop 1009: Bay / Edgewood>,
 <Stop 101: Haydenville IB>,
 <Stop 1012: State / Tilton (1)>,
 <Stop 1013: Liberty / Cleveland>,
 <Stop 1014: Main/Noble>,
 <Stop 1015: State / Tilton (2)>,
 <Stop 1017: Plainfield / Demond>,
 <Stop 1018: Elm / Lower Grandview (1)>,
 <Stop 102: Amherst Glass (Out)>,
 <Stop 1020: Parker / Sunrise (1)>,
 <Stop 1022: Carew/Armory>,
 <Stop 1023: Carew / Armory>,
 <Stop 1024: Plainfield / Laurel>,
 <Stop 1026: Main / Mechanic>,
 <Stop 1027: Court / Washington>,
 <Stop 1029: State / Ivanhoe (2)>,
 <Stop 103: Jones Library (Out)>,
 <Sto

In [9]:
sched.trips

[<Trip 3773017-N2205-VATCo-Weekday-01>,
 <Trip 3773018-N2205-VATCo-Weekday-01-1000000>,
 <Trip 3773019-N2205-VATCo-Weekday-01>,
 <Trip 3773020-N2205-VATCo-Weekday-01-1000000>,
 <Trip 3773021-N2205-VATCo-Weekday-01>,
 <Trip 3773022-N2205-VATCo-Weekday-01>,
 <Trip 3773023-N2205-VATCo-Weekday-01>,
 <Trip 3773024-N2205-VATCo-Weekday-01>,
 <Trip 3773025-N2205-VATCo-Weekday-01-1000000>,
 <Trip 3773026-N2205-VATCo-Weekday-01>,
 <Trip 3773027-N2205-VATCo-Weekday-01>,
 <Trip 3773028-N2205-VATCo-Weekday-01-1000000>,
 <Trip 3773029-N2205-VATCo-Weekday-01>,
 <Trip 3773030-N2205-VATCo-Weekday-01>,
 <Trip 3773031-N2205-VATCo-Weekday-01-1000000>,
 <Trip 3773032-N2205-VATCo-Weekday-01>,
 <Trip 3773033-N2205-VATCo-Weekday-01>,
 <Trip 3773034-N2205-VATCo-Weekday-01-1000000>,
 <Trip 3773035-N2205-VATCo-Weekday-01>,
 <Trip 3773036-N2205-VATCo-Weekday-01>,
 <Trip 3773037-N2205-VATCo-Weekday-01-1000000>,
 <Trip 3773038-N2205-VATCo-Weekday-01>,
 <Trip 3773039-N2205-VATCo-Weekday-01>,
 <Trip 3773040-N2205-VAT

In [10]:
sched.stop_times

C:\Users\Mahsa\Anaconda3\envs\oxe\lib\site-packages\pygtfs\schedule.py:52: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return instance_self.session.query(entity).all()


[<StopTime 3773017-N2205-VATCo-Weekday-01: 1>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 2>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 3>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 4>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 5>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 6>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 7>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 8>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 10>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 11>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 12>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 13>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 14>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 15>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 16>,
 <StopTime 3773017-N2205-VATCo-Weekday-01: 18>,
 <StopTime 3773018-N2205-VATCo-Weekday-01-1000000: 1>,
 <StopTime 3773018-N2205-VATCo-Weekday-01-1000000: 2>,
 <StopTime 3773018-N2205-VATCo-Weekday-01-1000000: 3>,
 <StopTime 3773018-N2205-VATCo-Weekday-01-1000000: 4>,
 <StopTime 3773018-N

In [11]:
sched.shapes

[<ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoint 300776>,
 <ShapePoi

In [23]:
import gtfs_functions as gtfs

In [24]:
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs("google_transit.zip")

ModuleNotFoundError: No module named 'partridge'